# Breakdown Metrics

In this notebook you will learn how to:

* Find breakdown metrics in an NVIDIA Nsight Compute report
* Extract and iterate over all its underlying metrics

## Setup

First, import NVIDIA Nsight Compute's Python Report Interface (PRI) as `ncu_report`
and load an `ncu-rep` report file with `load_report`.
We then extract a `kernel` using the subscript operator to get the first `IAction` of the first `IRange` of the `report`:

In [ ]:
import ncu_report

report_file_path = "../sample_reports/mergeSort.ncu-rep"
report = ncu_report.load_report(report_file_path)
kernel = report[0][0]

## Finding Breakdown Metrics

NVIDIA Nsight Compute supports many
[throughput metrics](https://docs.nvidia.com/nsight-"compute/ProfilingGuide/index.html#metrics-structure),
e.g. in the SOL section, for overall compute and memory throughput of a kernel.

The single value of a throughput metric is computed as the maximum of a _breakdown_ of underlying metrics.
The tool supports gathering these underlying metrics by collecting `breakdown:<throughput_metric_name>`.
For compute and memory throughput, this is already done as part of the SOL section.

To identify the underlying metric that gives the throughput its value, you need to look,
at the breakdown's constituents individually.
Breakdown metrics always contain the prefix "breakdown", which makes it easy to find them:

In [ ]:
for name in kernel:
    if "breakdown" in name:
        print(name)

For further information about the metric naming scheme please refer to the
[online documentation](https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#metrics-structure).

## Unpacking Breakdown Metrics

Having found a breakdown metric you can use its value to query all of its underlying metrics.
The value of a breakdown metric is a string of comma-separated metric names. You can obtain
the string by calling `value()` on the breakdown metric.

After this, you can use the individual metric names to extract the underlying metrics from `kernel`:

In [ ]:
breakdown_name = "breakdown:sm__throughput.avg.pct_of_peak_sustained_elapsed"
breakdown_metric = kernel[breakdown_name]
metric_names = breakdown_metric.value().split(",")

print(f"Breakdown of metric {breakdown_name}:")
for name in metric_names:
    print(f"  {name}: {kernel[name].value():.2f}%")